<a href="https://colab.research.google.com/github/chaeyeon2367/llm-langchain-pdf-chabot/blob/main/6_PDF_file_based_question_and_answer_chatbot_(Langchain%2C_Gradio%2C_ChatGPT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PDF file-based question and answer chatbot (Langchain, Gradio, ChatGPT)

In [2]:
!pip install openai
!pip install langchain
!pip install pinecone-client
!pip install gradio

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")
os.environ["PINECONE_ENV"] = getpass.getpass("Pinecone Environment:")

OpenAI API Key:··········
Pinecone API Key:··········
Pinecone Environment:··········


In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
import gradio as gr

In [5]:
!pip install pypdf
!pip install chromadb
!pip install tiktoken

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.vectorstores import Pinecone

In [8]:
loader = PyPDFLoader("/content/drive/MyDrive/data-llm/A State of the Art Review of large Generative AI models.pdf")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()

In [9]:
texts

[Document(page_content='ChatGPT is not all you need. A State of the Art\nReview of large Generative AI models\nRoberto Gozalo-Brizuela, Eduardo C. Garrido-Merch´ an\nQuantitative Methods Department, Universidad Pontiﬁcia Comillas, Madrid, Spain\n201905616@alu.comillas.edu, ecgarrido@icade.comillas.edu\nAbstract. During the last two years there has been a plethora of large\ngenerative models such as ChatGPT or Stable Diﬀusion that have been\npublished. Concretely, these models are able to perform tasks such as\nbeing a general question and answering system or automatically creat-\ning artistic images that are revolutionizing several sectors. Consequently,\nthe implications that these generative models have in the industry and\nsociety are enormous, as several job positions may be transformed. For\nexample, Generative AI is capable of transforming eﬀectively and cre-\natively texts to images, like the DALLE-2 model; text to 3D images,\nlike the Dreamfusion model; images to text, like the

In [27]:
import pinecone

# initialize pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

index_name = "langchainfundamentals"

# First, check if our index already exists. If it doesn't, we create it
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(name=index_name, metric="cosine", dimension=1536)
# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
docsearch = Pinecone.from_documents(texts, embeddings, index_name=index_name)

# if you already have an index, you can load it like this
# docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "What's a Generative AI?"
docs = docsearch.similarity_search(query)

In [22]:
index = pinecone.Index("langchainfundamentals")
vectorstore = Pinecone(index, embeddings.embed_query, "text")

vectorstore.add_texts("More text!")

/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


['b213b0ac-8a08-469f-9eb4-a422940088a7',
 '27128319-dcf2-4490-ae02-a8d864db07f5',
 'f4550b16-c88c-4b78-b56a-e962b714beb1',
 '8a341183-a5b1-4300-bcbb-985607133de3',
 'd49f2cb6-694e-4528-99fb-1d5c3c39ba13',
 '8b8b4067-414d-46e2-8b2d-7007b7fa44a3',
 '1d3593a2-ec55-419f-b5f6-90f191895162',
 '466fa811-4508-405e-a5b5-36a1ebc064f3',
 '945437cf-bd29-448f-a41f-8f50a1d97ac2',
 '097e8f3a-260e-4233-8509-62b4cf4d632c']

In [10]:
vector_store = Chroma.from_documents(texts, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True)

In [18]:
query = "What's a Generative AI?"
result = chain(query)
print(result)

{'question': "What's a Generative AI?", 'answer': "La Generative AI (Intelligence Artificielle Générative) fait référence à l'intelligence artificielle capable de générer du contenu nouveau, plutôt que d'analyser ou d'agir sur des données existantes comme le font les systèmes experts. Ces modèles d'intelligence artificielle générative sont capables de créer du contenu original en utilisant des modèles de discrimination ou de transformation entraînés sur de vastes ensembles de données. Ils peuvent générer des textes, des images, des vidéos, du code informatique, des textes scientifiques, etc. en fonction de leur architecture et des données sur lesquelles ils ont été formés. Ces modèles ont des implications énormes dans l'industrie et la société, car ils peuvent transformer de nombreux secteurs et métiers. [SOURCE]", 'sources': '', 'source_documents': [Document(page_content='ChatGPT is not all you need. A State of the Art\nReview of large Generative AI models\nRoberto Gozalo-Brizuela, Ed

In [19]:
result['answer']

"La Generative AI (Intelligence Artificielle Générative) fait référence à l'intelligence artificielle capable de générer du contenu nouveau, plutôt que d'analyser ou d'agir sur des données existantes comme le font les systèmes experts. Ces modèles d'intelligence artificielle générative sont capables de créer du contenu original en utilisant des modèles de discrimination ou de transformation entraînés sur de vastes ensembles de données. Ils peuvent générer des textes, des images, des vidéos, du code informatique, des textes scientifiques, etc. en fonction de leur architecture et des données sur lesquelles ils ont été formés. Ces modèles ont des implications énormes dans l'industrie et la société, car ils peuvent transformer de nombreux secteurs et métiers. [SOURCE]"

In [21]:
result['sources']

''

In [13]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""Use the following pieces of context to answer the users question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}

You MUST answer in French and in Markdown format:"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": prompt}

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
query = "Quel est le concept de Generative AI?"
result = chain(query)
print(result)

In [38]:
result['answer']

"La Generative AI fait référence à l'intelligence artificielle capable de générer du contenu nouveau et original, plutôt que d'analyser ou d'agir sur des données existantes. Ces modèles utilisent des architectures complexes et sont entraînés sur de vastes ensembles de données pour créer du contenu de manière autonome. Ils peuvent générer des images, du texte, de la musique, des vidéos, du code, etc. La Generative AI a des implications importantes dans de nombreux secteurs et peut transformer certains emplois. [SOURCE]"

In [39]:
result['source_documents']

[Document(page_content='State of the Art of Generative AI 3\nFig. 1. A taxonomy of the most popular generative AI models that have recently\nappeared classiﬁed according to their input and generated formats.\nFig. 2. Covered models by date of release. All models were released during 2022 except\nLaMDA, which was released in 2021 and Muse, in 2023.', metadata={'page': 2, 'source': '/content/drive/MyDrive/data-llm/A State of the Art Review of large Generative AI models.pdf', 'text': 'State of the Art of Generative AI 3\nFig. 1. A taxonomy of the most popular generative AI models that have recently\nappeared classiﬁed according to their input and generated formats.\nFig. 2. Covered models by date of release. All models were released during 2022 except\nLaMDA, which was released in 2021 and Muse, in 2023.'}),
 Document(page_content='ChatGPT is not all you need. A State of the Art\nReview of large Generative AI models\nRoberto Gozalo-Brizuela, Eduardo C. Garrido-Merch´ an\nQuantitative Meth

In [41]:
for doc in result['source_documents']:
    print('Content : ' + doc.page_content[0:100].replace('\n', ' '))
    print('File : ' + doc.metadata['source'])
    print('Page : ' + str(doc.metadata['page']))

Content : State of the Art of Generative AI 3 Fig. 1. A taxonomy of the most popular generative AI models that
File : /content/drive/MyDrive/data-llm/A State of the Art Review of large Generative AI models.pdf
Page : 2
Content : ChatGPT is not all you need. A State of the Art Review of large Generative AI models Roberto Gozalo-
File : /content/drive/MyDrive/data-llm/A State of the Art Review of large Generative AI models.pdf
Page : 0


In [17]:
def respond(message, chat_history):  # Define a function to handle responses from the chatbot.

    result = chain(message)

    bot_message = result['answer']

    # Append source document information to the bot's response
    for i, doc in enumerate(result['source_documents']):
        bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + '(' + str(doc.metadata['page']) + ') '

    chat_history.append((message, bot_message))  # Add the user's message and the bot's response to the chat history.

    return "", chat_history  # Return the modified chat history.

with gr.Blocks() as demo:  # Create an interface using gr.Blocks().
    chatbot = gr.Chatbot(label="Chat Window")  # Create a chatbot component with the label 'Chat Window'.
    msg = gr.Textbox(label="Input")  # Create a textbox with the label 'Input'.
    clear = gr.Button("Clear")  # Create a button with the label 'Clear'.

    # When a message is entered and submitted in the textbox, call the respond function.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    # When the 'Clear' button is clicked, reset the chat history.
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True)  # Run the interface. Upon execution, users can write messages in the 'Input' textbox, submit them, and clear the chat history using the 'Clear' button.

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://16837d11390d1ce188.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://16837d11390d1ce188.gradio.live
